In [1]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertForMaskedLM
import re
from sentence_transformers import SentenceTransformer, util
import heapq


2023-10-09 13:26:31.352962: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 13:26:31.375094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 13:26:31.678161: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 加载同义词集
cos_sim = np.load('cos_sim_counter_fitting.npy')

In [3]:
# 选择最相似的同义词
def pick_most_similar_words_batch(src_words, sim_mat, idx2word, ret_count=10, threshold=0.):
    """
    Given a list of source words (their indices), a similarity matrix, and an index-to-word mapping,
    this function returns the top `ret_count` similar words for each source word, filtered by a given threshold.
    
    Parameters:
    - src_words: List of source word indices.
    - sim_mat: Similarity matrix of shape (vocab_size, vocab_size).
    - idx2word: A mapping from word index to actual word.
    - ret_count: Number of top similar words to return for each source word.
    - threshold: A similarity threshold to filter out words.
    
    Returns:
    - sim_words: A list of lists containing similar words for each source word.
    - sim_values: A list of lists containing similarity values for each word in sim_words.
    """
    
    # 对于每个src_word，找到其与其他所有单词的相似度排名（从高到低）
    sim_order = np.argsort(-sim_mat[src_words, :])[:, 1:1 + ret_count]
    
    sim_words, sim_values = [], []  # 初始化列表以保存结果

    # 遍历src_words的每个词
    for idx, src_word in enumerate(src_words):
        # 获取对应src_word的相似度值
        sim_value = sim_mat[src_word][sim_order[idx]]
        
        # 根据阈值筛选出大于等于threshold的相似度值
        mask = sim_value >= threshold
        
        # 使用mask获取单词和其相似度值
        sim_word, sim_value = sim_order[idx][mask], sim_value[mask]
        
        # 将单词索引转换为实际的单词
        sim_word = [idx2word[id] for id in sim_word]
        
        # 保存结果
        sim_words.append(sim_word)
        sim_values.append(sim_value)

    return sim_words, sim_values  # 返回相似单词及其相似度值

In [4]:
# 计算影响最大的词
def influential_tokens(sentence, model, tokenizer, k=5):
    # 确保模型在评估模式
    model.eval()

    # 获取原始logit输出
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        original_logits = model(**inputs).logits

    # 为每个token计算logit差异
    token_ids = inputs["input_ids"][0].tolist()  # 获取token ids
    tokens = [tokenizer.decode([token_id]) for token_id in token_ids]  # 转换为tokens
    diffs = []

    for i, token_id in enumerate(token_ids):
        # 如果是[CLS], [SEP]或[PAD]，则跳过
        if token_id in [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]:
            continue

        # 将当前token替换为[MASK]
        masked_input_ids = inputs["input_ids"].clone()
        masked_input_ids[0][i] = tokenizer.mask_token_id

        # 获取mask后的logits
        with torch.no_grad():
            masked_logits = model(input_ids=masked_input_ids, attention_mask=inputs["attention_mask"]).logits

        # 计算logit差异
        diff = torch.abs(original_logits - masked_logits).sum().item()
        diffs.append((tokens[i], i, diff))  # 保存token, index和差异值

    # 根据差异大小排序tokens
    sorted_tokens = sorted(diffs, key=lambda x: x[2], reverse=True)

    # 返回前k个token及其索引
    return [(token_info[0], token_info[1]) for token_info in sorted_tokens[:k]]

In [5]:
# 正则表达式去掉特殊字符
def extract_words(sentence):
    words = re.findall(r'\b\w+\b', sentence)
    return ' '.join(words)

In [6]:
# 加载模型

# 窃取的模型
model_path = '/home/ubuntu/zhc/work/adversarial_output'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

sentence = "I drove by yesterday to get a sneak peak. It re-opens on July and I can't wait to take my kids. The new range looks amazing. The entire range appears to be turf, which may or many not help your game, but it looks really nice. The tee boxes look state of the art and the club house looks like something you'll see on a newer course. Can't wait to experience it!"
sentence = extract_words(sentence)
# print(len(sentence))
k = 2

top_k_tokens_and_indices = influential_tokens(sentence, model, tokenizer, k)

In [7]:
# topk个影响最大的词

top_k_tokens_and_indices

[('amazing', 28), ('nice', 48)]

In [8]:
def get_token_from_encoded(sentence, index, tokenizer):
    token_ids = tokenizer.encode(sentence, add_special_tokens=True)
    token = tokenizer.decode([token_ids[index]])
    return token


perturb_idxes = []
words_perturb = []
for i in range(k):
    perturb_idxes.append(top_k_tokens_and_indices[i][1])
    words_perturb.append(top_k_tokens_and_indices[i][0])


In [9]:
perturb_idxes, words_perturb #得到替换词下标和词

([28, 48], ['amazing', 'nice'])

In [10]:
idx2word = {}
word2idx = {}

print("Building vocab...")
with open('./counter-fitted-vectors.txt', 'r') as ifile:
    for line in ifile:
        word = line.split()[0]
        if word not in idx2word:
            idx2word[len(idx2word)] = word
            word2idx[word] = len(idx2word) - 1

Building vocab...


In [11]:
words_perturb_idx = [word2idx[word] for word in words_perturb if word in word2idx]
words_perturb_idx # 同义词典的位置

[7805, 25360]

In [12]:
words_perturb = [(idx, idx2word[idx]) for idx in words_perturb_idx]
words_perturb

[(7805, 'amazing'), (25360, 'nice')]

In [13]:
synonym_words, _ = pick_most_similar_words_batch(words_perturb_idx, cos_sim, idx2word, 200, 0.1)

In [14]:
synonym_words

[['stunning',
  'astounding',
  'impressive',
  'astonishing',
  'startling',
  'surprising',
  'striking',
  'staggering',
  'incredible',
  'breathtaking',
  'unbelievable',
  'awesome',
  'remarkable',
  'spectacular',
  'fantastic',
  'magnificent',
  'phenomenal',
  'dazzling',
  'marvelous',
  'extraordinary',
  'terrific',
  'fabulous',
  'superb',
  'marvellous',
  'sublime',
  'wonderful',
  'noteworthy',
  'splendid',
  'admirable',
  'wondrous',
  'excellent',
  'whopping',
  'gorgeous',
  'resplendent',
  'exceptional',
  'dramatic',
  'brilliant',
  'breathless',
  'sumptuous',
  'sensational',
  'exquisite',
  'uncanny',
  'extraordinaire',
  'funky',
  'beautiful',
  'tremendous',
  'impressed',
  'formidable',
  'magnifique',
  'outstanding',
  'fantastical',
  'glamorous',
  'great',
  'stupendous',
  'delightful',
  'prodigious',
  'memorable',
  'super',
  'ravishing',
  'glorious',
  'mesmerizing',
  'handsome',
  'supernatural',
  'unthinkable',
  'stun',
  'lovely

### compute M1

In [15]:
def mask_sentence(sentence, mask_idx, tokenizer):
    """Inserts a [MASK] token at the specified index of the sentence."""
    tokens = tokenizer.tokenize(sentence)
    tokens[mask_idx] = '[MASK]'
    return tokenizer.convert_tokens_to_string(tokens)

In [17]:
model_name = "bert-base-uncased"
mlm_model = BertForMaskedLM.from_pretrained(model_name)
mlm_tokenizer = BertTokenizer.from_pretrained(model_name)
mlm_model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [23]:
word_mask_probability_list = []

for index, idx in enumerate(perturb_idxes):    
    msk_sentence = mask_sentence(sentence, mask_idx=idx-1, tokenizer=mlm_tokenizer)

    print(idx,msk_sentence)
    
    inputs = mlm_tokenizer(msk_sentence, return_tensors="pt")
    mask_idx = torch.where(inputs["input_ids"][0] == mlm_tokenizer.mask_token_id)[0].item()
    # 3. 使用BERT预测[MASK]位置的概率分布
    with torch.no_grad():
        outputs = mlm_model(**inputs)
        predictions = outputs.logits[0, mask_idx].softmax(dim=0)

    word_mask_probability = {}
    
    for word in synonym_words[index]:
        word_id = tokenizer.convert_tokens_to_ids(word)
        word_probability = predictions[word_id].item()
        word_mask_probability[word] = word_probability
        # print(word_mask_probability)
    # print("---")
    
    total_value = sum(word_mask_probability.values())
    normalized_data = {k: v / total_value for k, v in word_mask_probability.items()}

    word_mask_probability_list.append(normalized_data)   

28 i drove by yesterday to get a sneak peak it re opens on july and i can t wait to take my kids the new range looks [MASK] the entire range appears to be turf which may or many not help your game but it looks really nice the tee boxes look state of the art and the club house looks like something you ll see on a newer course can t wait to experience it
48 i drove by yesterday to get a sneak peak it re opens on july and i can t wait to take my kids the new range looks amazing the entire range appears to be turf which may or many not help your game but it looks really [MASK] the tee boxes look state of the art and the club house looks like something you ll see on a newer course can t wait to experience it


### compute M2 

In [25]:

def replace_token_at_index(sentence, index, replacement_word, tokenizer):
    token_ids = tokenizer.encode(sentence, add_special_tokens=False)
    replacement_ids = tokenizer.encode(replacement_word, add_special_tokens=False)
    
    if len(replacement_ids) != 1:
        # 将replacement_word设置为"[UNK]"标记
        replacement_word = "[UNK]"
        replacement_ids = tokenizer.encode(replacement_word, add_special_tokens=False)
    
    token_ids[index] = replacement_ids[0]
    return tokenizer.decode(token_ids)

def generate_nested_sentences(sentence, indices, replacement_words_nested_list, tokenizer):
    if len(indices) != len(replacement_words_nested_list):
        raise ValueError("Length of indices and replacement words nested list should be the same.")
    
    nested_sentences = []
    for idx, replacement_words_list in zip(indices, replacement_words_nested_list):
        modified_sentences_for_idx = []
        for replacement_word in replacement_words_list:
            modified_sentence = replace_token_at_index(sentence, idx, replacement_word, tokenizer)
            modified_sentences_for_idx.append(modified_sentence)
        nested_sentences.append(modified_sentences_for_idx)
    return nested_sentences


indices = perturb_idxes
replacement_words_nested_list = synonym_words

nested_sentences = generate_nested_sentences(sentence, indices, replacement_words_nested_list, tokenizer)


In [26]:

similar_model = SentenceTransformer('bert-base-nli-mean-tokens')


word_similarity_list = []
# 对单一句子进行编码
query_embedding = similar_model.encode(sentences=sentence, convert_to_tensor=True)

for i in range(len(nested_sentences)):
    # 对多个句子进行编码
    sentence_embeddings = similar_model.encode(nested_sentences[i], convert_to_tensor=True)
        
    # 计算相似度
    similarity_scores = util.pytorch_cos_sim(query_embedding, sentence_embeddings)
    similarity_scores = similarity_scores.to('cpu').numpy().reshape(-1)
    
    word_similarity_probability = {}
    for idx, word in enumerate(synonym_words[i]):    # break
        word_similarity_probability[word] = similarity_scores[idx]
    total_value = sum(word_similarity_probability.values())
    normalized_data = {k: v / total_value for k, v in word_similarity_probability.items()}

    word_similarity_list.append(normalized_data)   

### compute M3

In [28]:
# 加载微调后的BERT模型
MODEL_PATH = "/home/ubuntu/zhc/work/adversarial_output"
adver_tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
adver_model = BertForSequenceClassification.from_pretrained(MODEL_PATH)

# 计算Logit输出
def compute_logits(sentence):
    inputs = adver_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = adver_model(**inputs)
    logits = outputs.logits
    return logits

# 计算概率输出
def compute_probabilities(sentence):
    logits = compute_logits(sentence)
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    return probabilities

# 计算M3概率矩阵
def compute_difference_matrix(original_sentence, nested_adversarial_sentences, nested_replacement_words):
    difference_matrix = []

    # 为原始句子计算softmax概率
    original_probs = compute_probabilities(original_sentence)
    #print(original_probs)
    original_pred_class = torch.argmax(original_probs, dim=1).item()
    original_pred_prob = original_probs[0, original_pred_class].item()

    for group_idx, group in enumerate(nested_adversarial_sentences):
        group_differences = {}

        for sentence_idx, sentence in enumerate(group):
            adversarial_probs = compute_probabilities(sentence)
            #print(adversarial_probs)

            adversarial_pred_class = torch.argmax(adversarial_probs, dim=1).item()
            adversarial_pred_prob = adversarial_probs[0, original_pred_class].item()
            #print(adversarial_pred_prob)

            # 计算概率差值
            difference = original_pred_prob - adversarial_pred_prob

            # 使用替换词作为key存储差值
            replacement_word = nested_replacement_words[group_idx][sentence_idx]
            group_differences[replacement_word] = difference
            #print(difference)
            # break
        difference_matrix.append(group_differences)
    return difference_matrix


# 假设你已经有一个嵌套的对抗句子列表
original_sentence = sentence

nested_adversarial_sentences = nested_sentences

# 假设你有一个替换词列表与上面的对抗句子列表相对应
nested_replacement_words = synonym_words


difference_matrix = compute_difference_matrix(original_sentence, nested_adversarial_sentences, nested_replacement_words)
print(difference_matrix)

[{'stunning': 3.30805778503418e-05, 'astounding': 5.0067901611328125e-06, 'impressive': 2.4497509002685547e-05, 'astonishing': 1.52587890625e-05, 'startling': 2.580881118774414e-05, 'surprising': 1.4543533325195312e-05, 'striking': 3.2842159271240234e-05, 'staggering': 3.0100345611572266e-05, 'incredible': 1.5616416931152344e-05, 'breathtaking': 5.0067901611328125e-06, 'unbelievable': 1.049041748046875e-05, 'awesome': 1.901388168334961e-05, 'remarkable': 1.4781951904296875e-05, 'spectacular': 1.8596649169921875e-05, 'fantastic': 1.52587890625e-05, 'magnificent': 3.2007694244384766e-05, 'phenomenal': 5.0067901611328125e-06, 'dazzling': 2.7000904083251953e-05, 'marvelous': 1.2755393981933594e-05, 'extraordinary': 2.4378299713134766e-05, 'terrific': 1.8835067749023438e-05, 'fabulous': 2.568960189819336e-05, 'superb': 2.6166439056396484e-05, 'marvellous': 5.0067901611328125e-06, 'sublime': 1.996755599975586e-05, 'wonderful': 1.049041748046875e-05, 'noteworthy': -8.344650268554688e-07, 'spl

In [48]:
EPSILON = 1e-10  # 这是一个非常小的数，您可以根据需要调整它

def get_global_min(lst):
    """获取所有字典中的全局最小值"""
    global_min = float('inf')
    for d in lst:
        local_min = min(d.values())
        global_min = min(global_min, local_min)
    return global_min

def l1_normalize_dict(d, shift_value):
    """对偏移后的字典进行L1归一化"""
    shifted_values = [v + shift_value for v in d.values()]
    total = sum(shifted_values)
    
    if total == 0:
        return {k: 1.0/len(d) for k in d}

    normalized_dict = {k: shifted_val / total for k, shifted_val in zip(d.keys(), shifted_values)}
    
    # 替换为0的值
    for k, v in normalized_dict.items():
        if v == 0:
            normalized_dict[k] = EPSILON
            
    return normalized_dict

def l1_normalize_list_of_dicts(lst):
    """对整个列表的字典进行L1归一化"""
    global_min = get_global_min(lst)
    shift_value = -global_min if global_min < 0 else 0
    return [l1_normalize_dict(d, shift_value) for d in lst]

# 示例：
normalized_lst = l1_normalize_list_of_dicts(difference_matrix)
normalized_lst

[{'stunning': 0.008974057528019438,
  'astounding': 0.0028215377380672465,
  'impressive': 0.007093032369307939,
  'astonishing': 0.0050683177887504245,
  'startling': 0.0073804112129999735,
  'surprising': 0.004911565692191133,
  'striking': 0.00892180682916634,
  'staggering': 0.008320923792355724,
  'incredible': 0.00514669383703007,
  'breathtaking': 0.0028215377380672465,
  'unbelievable': 0.004023303811688481,
  'awesome': 0.005891266295686705,
  'remarkable': 0.00496381639104423,
  'spectacular': 0.005799827572693785,
  'fantastic': 0.0050683177887504245,
  'magnificent': 0.0087389293831805,
  'phenomenal': 0.0028215377380672465,
  'dazzling': 0.00764166470726546,
  'marvelous': 0.004519685450792905,
  'extraordinary': 0.007066907019881391,
  'terrific': 0.005852078271546882,
  'fabulous': 0.007354285863573425,
  'superb': 0.00745878726127962,
  'marvellous': 0.0028215377380672465,
  'sublime': 0.006100269091099094,
  'wonderful': 0.004023303811688481,
  'noteworthy': 0.00154139

### M1->word_similarity_list M2->word_mask_probability_list M3->normalized_lst

In [50]:
import math 

combined_probs = []

# 添加dict3到zip中
for dict1, dict2, dict3 in zip(word_similarity_list, word_mask_probability_list, normalized_lst): 
    combined_dict = {}
    for token in dict1:
        # 对三个字典中的对应值求和
        combined_dict[token] = math.log(dict1[token]) + math.log(dict2[token]) + math.log(dict3[token])
    combined_probs.append(combined_dict)


In [51]:
combined_probs

[{'stunning': -13.775617825919053,
  'astounding': -20.928048984983338,
  'impressive': -12.150611916490174,
  'astonishing': -18.861023272576666,
  'startling': -19.535171145057703,
  'surprising': -18.791820429742288,
  'striking': -18.73003487547863,
  'staggering': -20.443333500841575,
  'incredible': -13.570005354415427,
  'breathtaking': -20.928048984983338,
  'unbelievable': -16.459270230541545,
  'awesome': -12.887230983846425,
  'remarkable': -16.72601091134727,
  'spectacular': -14.885460050870869,
  'fantastic': -13.663795799195423,
  'magnificent': -14.379587259478635,
  'phenomenal': -20.928048984983338,
  'dazzling': -18.536307862344113,
  'marvelous': -18.286507326983255,
  'extraordinary': -17.448030015442725,
  'terrific': -16.437503807888938,
  'fabulous': -15.361477629154276,
  'superb': -18.291062786563472,
  'marvellous': -20.928048984983338,
  'sublime': -20.917673422713403,
  'wonderful': -14.139722255552947,
  'noteworthy': -22.318115980278684,
  'splendid': -17

### 攻击，启动！

In [52]:
def replace_tokens_in_sentence(sentence, indices, replacement_tokens, tokenizer):
    tokens = tokenizer.tokenize(sentence)
    for i, index in enumerate(indices):
        tokens[index-1] = replacement_tokens[i]
    return tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens))

In [53]:
def predict_sentiment(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to('cuda') for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

In [54]:
black_box_path = "/home/ubuntu/zhc/work/output"
tokenizer = BertTokenizer.from_pretrained(black_box_path)
model = BertForSequenceClassification.from_pretrained(black_box_path)
model.eval()
model.to('cuda')  # if you are using GPU
original_sentiment = predict_sentiment(sentence, model, tokenizer)
original_sentiment

1

In [55]:

def k_best_viterbi(probs, k=1):
    paths = [(score, [token]) for token, score in probs[0].items()]
    for i in range(1, len(probs)):
        new_paths = []
        for prev_score, prev_path in paths:
            for next_token, next_score in probs[i].items():
                new_score = prev_score + next_score
                heapq.heappush(new_paths, (new_score, prev_path + [next_token]))
                if len(new_paths) > k:
                    heapq.heappop(new_paths)
        paths = new_paths
    return sorted(paths, key=lambda x: x[0], reverse=True)

In [56]:

max_attempts = 10000
success = False
attempted_paths = []

for attempt in range(1, max_attempts+1):
    all_paths = k_best_viterbi(combined_probs, k=max_attempts)
    best_path_not_attempted = next((path for path in all_paths if path[1] not in attempted_paths), None)

    if not best_path_not_attempted:
        break

    score, path = best_path_not_attempted
    attempted_paths.append(path)
    modified_sentence = replace_tokens_in_sentence(sentence, indices, path, tokenizer)
    modified_sentiment = predict_sentiment(modified_sentence, model, tokenizer)

    if modified_sentiment != original_sentiment:
        print(f"Attack successful with sentence: {modified_sentence}")
        success = True
        break
    print(attempt)
    # 降低已经尝试路径的得分，以便在下次迭代中不再选择它们
    for idx, token in enumerate(path):
        combined_probs[idx][token] -= 1e6  # 使其得分显著降低

if not success:
    print("Attack failed after trying all paths.")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
